In [4]:
%pip install torch transformers diffusers accelerate torchvision

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 KB 955.1 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 86.6 kB/s eta 0:00:0000:0100:03m
Note: you may need to restart the kernel to use updated packages.


In [9]:
import torch
from diffusers import StableDiffusionPipeline

# Load the pre-trained model
model_id = "CompVis/stable-diffusion-v1-4"
pipeline = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Loading pipeline components...: 100%|██████████| 7/7 [00:06<00:00,  1.01it/s]


In [10]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class CustomImageDataset(Dataset):
    def __init__(self, img_dir, watermark_path, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.img_names = os.listdir(img_dir)
        self.watermark = Image.open(watermark_path).convert("RGBA")
    
    def __len__(self):
        return len(self.img_names)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.watermark

# Define transformations
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Load dataset
img_dir = "./images"
watermark_path = "./watermark.png"
dataset = CustomImageDataset(img_dir, watermark_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [11]:
from torchvision.transforms.functional import to_pil_image, to_tensor

def embed_watermark(image, watermark, alpha=0.5):
    image = to_pil_image(image)
    watermark = watermark.resize(image.size, Image.ANTIALIAS)
    
    image.paste(watermark, (0, 0), watermark)
    
    return to_tensor(image)

In [12]:
from torch import optim
from diffusers import DDPMScheduler

# Initialize optimizer and scheduler
optimizer = optim.AdamW(pipeline.unet.parameters(), lr=1e-5)
scheduler = DDPMScheduler()

num_epochs = 5  # Adjust based on your needs

for epoch in range(num_epochs):
    for images, watermark in dataloader:
        images = images.to("cuda")
        
        # Embed watermark in images
        watermarked_images = torch.stack([embed_watermark(img, watermark, alpha=0.5) for img in images])
        
        # Generate noise
        noise = torch.randn_like(watermarked_images).to("cuda")
        
        # Get model prediction and calculate loss
        noise_pred = pipeline.unet(watermarked_images, noise)
        loss = torch.nn.functional.mse_loss(noise_pred, noise)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch + 1}/{num_epochs} completed. Loss: {loss.item()}")

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [ ]:
pipeline.save_pretrained("path/to/save/fine-tuned-model")

In [ ]:
from diffusers import StableDiffusionPipeline

# Load the fine-tuned model
fine_tuned_model_id = "path/to/save/fine-tuned-model"
fine_tuned_pipeline = StableDiffusionPipeline.from_pretrained(fine_tuned_model_id, torch_dtype=torch.float16).to("cuda")

# Generate images
prompt = "a beautiful landscape painting"
generated_image = fine_tuned_pipeline(prompt).images[0]

# Save or display the generated image
generated_image.save("generated_image.png")
generated_image.show()